<span style="color:maroon">

# Extract traits from legume images
    
## =====================================================================

## Legumes Example
## =====================================================================

    
</span>


We want to extract traits about different legumes. We will measure these traits using image analysis and save data into a CSV for Machine Learning in a later activity.

<span style="color:purple">

Headers in purple will indicate a step that **might** need adjusting to parameterize the workflow to your particular image.
    
</span>


## June 2024

In [ ]:
# Import libraries
%matplotlib widget
from plantcv import plantcv as pcv

In [ ]:
# Print out the version of PlantCV being used by the Jupyter kernel
pcv.__version__


# Initialize workflow inputs & outputs

In [ ]:
# Set debugging parameters
pcv.params.debug = "plot"
pcv.params.text_size = 25
pcv.params.text_thickness = 25



What exactly is a filepath? In general, a path is a string of characters which specifies a unique location in a directory or page hierarchy. For file systems, each level in the hierarchy is a directory.

`/home/user/python/test.py`

In this file path, the test.py file is inside the python directory. The python directory is a subdirectory of the user directory, which is a subdirectory of the home directory. Absolute file paths specify the location of a file from the root directory in the file system structure. They are also called “full file paths” or “full paths.” In Linux, the tilde (~) is commonly used to represent a user’s home directory in a file path. Relative file paths specify the location of a file in the same folder or on the same server. In other words, a relative file path specifies a location of a file that is relative to the current directory.


(https://www.codecademy.com/resources/docs/general/file-paths)

<span style="color:purple">


## Read in the image
    
File name and extension are case sensitive.

</span>


In [ ]:
# Read image

# Inputs:
#   filename - Image file to be read in
#   mode - How to read in the image; either 'native' (default), 'rgb', 'gray', or 'csv'

img, imgpath, imgname = pcv.readimage(filename="")



<span style="color:purple">
    
# Rename your seed

### Use CamelCase and
## avoid spaces or underscores !!!

</span>

In [ ]:
seed_name = 'bean_name' # Change this to the name of the bean you imaged!!

In [ ]:
seed_name

## Detecting and Calibrating Colors with Color Cards/Color Checkers

In [ ]:
# Load your rotated or cropped image into the rgb_img argument to detect the color card.
cc_mask = pcv.transform.detect_color_card(rgb_img=img)

# We will also print the average chip size and store the values in outputs.observations.
print(pcv.outputs.observations['default']['median_color_chip_size']['value'])

In [ ]:
# Next, we make a color card matrix. You will not see an output for this step.

headers, card_matrix = pcv.transform.get_color_matrix(rgb_img=img, mask=cc_mask)

In [ ]:
# Define the standard color card matrix, we know what the colors of those chips should be in a "perfect" image, so we will correct to those values
# Look at where your white chip is in the image to determine which position your card is in (pos)
# When using detect_color_card, you will always set pos=3.

#pos     = reference value indicating orientation of the color card. The reference
       #         is based on the position of the white chip:
        #        pos = 0: bottom-left corner
        #        pos = 1: bottom-right corner
        #        pos = 2: top-right corner
        #        pos = 3: top-left corner

std_color_matrix = pcv.transform.std_color_matrix (pos=3)

In [ ]:
#Color correct your image to the standard values.
#look at the image - does the color look good? 
# If it looks crazy, you probably don't have the card found well and need to go back to 
# define the start and spacing for the card.

img_cc = pcv.transform.affine_color_correction(img, card_matrix, std_color_matrix)

## Visualize Colorspaces
The visualization tool converts the color image into HSV and LAB colorspaces and displays the grayscale channels in a matrix so that they can be visualized simultaneously. The idea is to select a channel that maximizes the difference between the plant and the background pixels.

In [ ]:
# Inputs:
#   rbg_img      = original image
#   original_img = whether to include the original RGB images in the display: True (default) or False

all_c = pcv.visualize.colorspaces(rgb_img=img_cc, original_img=False)


## Convert the color image to grayscale
Converts the input color image into the LAB colorspace and returns the B (blue-yellow) channel as a grayscale image. We have already tested ever type of bean and found that "b" channel did well (since we chose a blue background).

In [ ]:
# Inputs:
#   rbg_img - original image
#   channel - desired colorspace ('l', 'a', or 'b')

gray = pcv.rgb2gray_(rgb_img=img_cc, channel="")


# Visualize the distribution of grayscale values
A histogram can be used to visualize the distribution of values in an image. The histogram can aid in the selection of a threshold value.


In [ ]:
# Inputs:
#   img         = gray image in selected colorspace
#   mask        = None (default), or mask
#   bins        = 100 (default) or number of desired number of evenly spaced bins
#   lower-bound = None (default) or minimum value on x-axis
#   upper-bound = None (default) or maximum value on x-axis
#   title       = None (default) or custom plot title
#   hist_data   = False (default) or True (if frequency distribution data is desired)

hist = pcv.visualize.histogram(img=gray, bins=30)


## Threshold the grayscale image


In [ ]:
bin_mask = pcv.threshold.binary(gray_img=gray, threshold= , object_type="")

<span style="color:purple">


## Remove small background noise
    
_Thresholding mostly labeled plant pixels white but also labeled small regions of the background white. The fill function removes "salt" noise from the background by filtering white regions by size. The resolution of the image will factor into the average object sizes in your images so this step might require adjustment._

</span>


In [ ]:
# Inputs:
#   bin_img - binary mask image
#   size - maximum size for objects that should be filled in as background (non-plant) pixels

fill = pcv.fill(bin_img=bin_mask, size=)
#                                       /\
#                                       |
#                                 change this value (maybe, mostly depends on img size)


## Flood fill "pepper" noise

The `pcv.fill_holes` function does a flood fill of any missing portions that are surrounded by white pixels. This will address the glare in the center of each bean.

In [ ]:
# Inputs:
#   bin_img - binary mask image

clean_mask = pcv.fill_holes(bin_img=fill)


<span style="color:purple">
    
# Define Region of Interest    
    
_Highly likely that this step will need the parameters adjusted to each image_

</span>


In [ ]:
# Inputs:
#   img         = RGB or grayscale image for plotting
#   x           = x coordinate of the center of ROI
#   y           = y coordinate of the center of ROI
#   r           = radium of the ROI to get drawn


roi = pcv.roi.rectangle(img=img_cc, x=, y=, h=, w=)


In [ ]:
# Inputs:
#   mask         = Binary image
#   roi          = Region of interest, defined in an upstream step
#   roi_type     = 'cutto', 'partial' (for partially inside, default), or
#                  'largest' (keep only the largest contour)

filtered_mask = pcv.roi.filter(mask=clean_mask, roi=roi, roi_type="partial")


## Investigate object sizes

In [ ]:
pcv.params.text_size = 1
pcv.params.text_thickness = 2

# Inputs:
#   img         = gray image in selected colorspace
#   mask        = None (default), or mask
#   num_objects = Optional parameter to limit the number of objects that will get annotated (default = 100).

sizes = pcv.visualize.obj_sizes(img=img, mask=filtered_mask, num_objects=100)


Salt & pepper noise are small white/black pixels, respectively, in the binary mask. In this example image, the flash creates a glare that makes the centers of the beans get excluded during segmentation, but we can recover these pixels with some clean up.

## Create labeled mask
We want to extract traits from each bean replicate, so we need to create a mask that has unique pixel values for each identified object.

In [ ]:
# Inputs:
#    mask            = mask image
#    rois            = (Optional) list of multiple ROIs (from roi.multi or roi.auto_grid)
#    roi_type        = (Optional)''partial' (for partially inside, default), cutto' (hard cut at boundary),
#                      'largest' (keep only the largest contour)

labeled_mask, num = pcv.create_labels(mask=filtered_mask)


## Extract seed shape and color traits


In [ ]:
# Extract size traits

# Inputs:
        #   img          = RGB image for debugging
        #   labeled_mask = Grayscale mask with unique pixel value per object of interest
        #   n_labels     = Total number expected individual objects (default = 1).
        #   label        = Modifies the variable name of observations recorded (default = "default").

shape_img = pcv.analyze.size(img=img_cc, labeled_mask=labeled_mask, n_labels=num, label=seed_name)


# Extract color data!

In [ ]:
# Extract color traits from each replicate

# Inputs:
        #   img          = RGB image for debugging
        #   labeled_mask = Grayscale mask with unique pixel value per object of interest
        #   n_labels     = Total number expected individual objects (default = 1).
        #   colorspaces  = 'all', 'rgb', 'lab', or 'hsv' (default = 'hsv')
        #   label        = Modifies the variable name of observations recorded (default = "default").
color_img = pcv.analyze.color(rgb_img=img_cc, labeled_mask=labeled_mask, n_labels=num, colorspaces="hsv",
                              label=seed_name)


In [ ]:
# Look at what has been stored into the Outputs class as observations from our workflow
# Pretty unreadable due to the heirarchical format, but we just extracted TONS of raw phenotype data

#pcv.outputs.observations

# How large is the first bean?

In [ ]:
# Index the dictionary of traits to look at the area for one replicate

indexing_name = f"{seed_name}_1"
pcv.outputs.observations[indexing_name]['area']['value']


## Save results

During analysis, measurements are stored in the background in the outputs recorder.

This example includes image analysis for 'area', 'convex_hull_area', 'solidity', 'perimeter', 'width', 'height', 'longest_path', 'center_of_mass, 'convex_hull_vertices', 'object_in_frame', 'ellipse_center', 'ellipse_major_axis', 'ellipse_minor_axis', 'ellipse_angle', 'ellipse_eccentricity' using `pcv.analyze.size` and color analysis using `pcv.analyze.color`.

Here, results are saved to a CSV file. Filename will update with bean name set at top of this workflow.

In [ ]:
csv_filename = f"{seed_name}_csv.csv"
pcv.outputs.save_results(csv_filename, "csv")


Now we can go look for the CSV file that we just saved out.

<span style="color:maroon">

# Duplicate this workflow

_Click the (File) tab in the top left corner. (Make a copy ... ) And rename the new jupyter notebook with your next bean type._
    
</span>